In [1]:
# pip install joblib
# pip install nibabel 
# !pip install matplotlib
# !pwd

# !pip install scikit-image
# !pip install plotly


import os
import nibabel as nib
import numpy as np

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"): # Check for specific image file extensions
            img_path = os.path.join(folder_path, filename)
            img = nib.load(img_path)
            img_data = img.get_fdata()
            print(f"Shape of image {filename}: {img_data.shape}")
            images.append(img_data)
    return np.array(images)

# Example usage:
path = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'
folder_path = path # Replace with the path to your folder
loaded_images = load_images_from_folder(folder_path)

Shape of image sub-A00000541_ses-20100101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000838_ses-20100101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000865_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)


In [2]:
for i in loaded_images:
    print(i.shape)    

(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)


In [4]:
len(loaded_images)
!pwd
# /mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again

/mnt/md0/cads-phd/explainbrainROI


In [5]:
# !pip install tqdm

In [ ]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed


directory = '/mnt/md0/cads-phd/For_SPR/mini_data' 

# Running FSL ANAT and getting FSL ANAT Folders 

# n_jobs  = -1 # Number of jobs to run in parallel. -1 means use all available processors.
n_jobs = 10

def process_anat_volumes(directory, n_jobs=6):
    """
    Process anatomical volumes for files in a specified directory using parallel computing.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    process_anat_volumes('/path/to/your/directory/', n_jobs=8)
    """
    def anat_volumes(filename):
        full_path = os.path.join(directory, filename)
        return subprocess.run(["fsl_anat", "-i", full_path], capture_output=True)

    Parallel(n_jobs=n_jobs)(delayed(anat_volumes)(filename) for filename in os.listdir(directory))

process_anat_volumes(directory, n_jobs=n_jobs)




In [ ]:

# upon running the above which is a function, it will return a subprocess.CompletedProcess object containing information about the completed process.
# a folder with various files will be created in the directory where the function is run.  This folder will be named after the file that was processed and will contain the following files:
#  the naming will be example_patient.nii.gz --> example_patient.anat



In [4]:
# import subprocess
# import os
# import time
# import glob
# from joblib import Parallel, delayed

# directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'

# def invwarp(directory, ref_file, warp_file, out_file_name, verbose=False):
#     """
#     Run the FSL command 'invwarp' to invert a non-linear warp field.

#     Parameters:
#     directory (str): The directory path where the files are located.
#     ref_file (str): The reference file in the specified directory.
#     warp_file (str): The warp file in the specified directory.
#     out_file_name (str): The name of the output file.
#     verbose (bool): If True, display verbose output (default is False).

#     Returns:
#     CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

#     Example:
#     invwarp('/path/to/your/directory/', 'T1.nii.gz', 'T1_to_MNI_nonlin_field.nii.gz', 'std_subject_space', verbose=True)
#     """
#     ref_path = f"{directory}/{ref_file}"
#     warp_path = f"{directory}/{warp_file}"
#     out_path = f"{directory}/Warps/{out_file_name}"
#     verbose_arg = "--verbose" if verbose else ""

#     return subprocess.run(["invwarp", f"--ref={ref_path}", f"--warp={warp_path}", f"--out={out_path}", verbose_arg], capture_output=True)


# # For the Cortical Regions- pulling from FSL_Anat Folder 



# def applywarp_cort(directory, ref_file, in_file, warp_file, out_file_name, interp_method='nn'):
#     """
#     Apply a warp to an input file and resample it to the space of a reference file.

#     Parameters:
#     directory (str): The directory path where the files are located.
#     ref_file (str): The reference file in the specified directory.
#     in_file (str): The input file to be warped.
#     warp_file (str): The warp file in the specified directory.
#     out_file_name (str): The name of the output file.
#     interp_method (str): The interpolation method to be used (default is 'nn').

#     Returns:
#     CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

#     Example:
#     applywarp_cort('/path/to/your/directory/', 'T1.nii.gz', 'HarvardOxford-cort-maxprob-thr50-2mm.nii.gz', 'std_subject_space.nii.gz', 'HO_in_subj_t1_space.nii.gz', interp_method='trilinear')
#     """
#     ref_path = f"{directory}/{ref_file}"
#     in_path = f"/usr/local/fsl/data/atlases/HarvardOxford/{in_file}"
#     warp_path = f"{directory}/Warps/{warp_file}"
#     out_path = f"{directory}/Warps/{out_file_name}"

#     return subprocess.run(["applywarp", f"--ref={ref_path}", f"--in={in_path}", f"--warp={warp_path}", f"--out={out_path}", f"--interp={interp_method}"], capture_output=True)



# def applywarp_subcort(directory, ref_file, in_file, warp_file, out_file_name, interp_method='nn'):
#     """
#     Apply a warp to an input file and resample it to the space of a reference file.

#     Parameters:
#     directory (str): The directory path where the files are located.
#     ref_file (str): The reference file in the specified directory.
#     in_file (str): The input file to be warped.
#     warp_file (str): The warp file in the specified directory.
#     out_file_name (str): The name of the output file.
#     interp_method (str): The interpolation method to be used (default is 'nn').

#     Returns:
#     CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

#     Example:
#     applywarp_subcort('/path/to/your/directory/', 'T1.nii.gz', 'HarvardOxford-sub-maxprob-thr50-2mm.nii.gz', 'std_subject_space.nii.gz', 'subcort_HO_in_subj_t1_space.nii.gz', interp_method='trilinear')
#     """
#     ref_path = f"{directory}/{ref_file}"
#     in_path = f"/usr/local/fsl/data/atlases/HarvardOxford/{in_file}"
#     warp_path = f"{directory}/Warps/{warp_file}"
#     out_path = f"{directory}/Warps/{out_file_name}"

#     return subprocess.run(["applywarp", f"--ref={ref_path}", f"--in={in_path}", f"--warp={warp_path}", f"--out={out_path}", f"--interp={interp_method}"], capture_output=True)


# def apply_warps(path):
#     """
#     Apply warps to specified files and save the results in the 'Warps' folder.

#     Parameters:
#     path (str): The directory path where the files are located.

#     Returns:
#     None

#     Example:
#     apply_warps('/path/to/your/directory/')
#     """
#     folder = os.path.join(path, "Warps") 
#     if not os.path.exists(folder):
#         os.mkdir(folder)

#     invwarp(path)
#     time.sleep(3)  # Sleep for 3 seconds to ensure completion before the next step
#     applywarp_cort(path)
#     applywarp_subcort(path)



# def apply_warps_parallel(directory, n_jobs=6):
#     """
#     Apply warps to files in a specified directory in parallel.

#     Parameters:
#     directory (str): The directory path where the files are located.
#     n_jobs (int): The number of jobs to run in parallel (default is 6).

#     Returns:
#     None

#     Example:
#     apply_warps_parallel('/path/to/your/directory/', n_jobs=8)
#     """
#     def apply_warps(path):
#         folder = os.path.join(path, "Warps") 
#         if not os.path.exists(folder):
#             os.mkdir(folder)

#         invwarp(path)
#         time.sleep(3)  # Sleep for 3 seconds to ensure completion before the next step
#         applywarp_cort(path)
#         applywarp_subcort(path)

#     Parallel(n_jobs=n_jobs)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))



# apply_warps_parallel(directory, n_jobs=15)
# #questions to be asked

# # How long does this take for each image? 
# # How long does this take for all images?
    
# # What folders does this pull from?
# # Do i need to move folders? 

# # What is the output?


# # What function do I need to run to get a total output? 


In [2]:
# import subprocess
# import os
# # import time
# import glob
# from joblib import Parallel, delayed



# directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'  #to practice on how pipeline will run with multiple images 

# def invwarp(directory):
#     return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)


# def applywarp_cort(directory):
#     return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
#                "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
#                "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

# def applywarp_subcort(directory):
#     return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
#                "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
#                "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)


# def apply_warps(path):
    
#     folder = os.path.join(path, "Warps") 
#     if not os.path.exists(folder):
#         os.mkdir(folder)

#     invwarp(path)
#     print(path + ' invwarp done')
#     time.sleep(3)
#     applywarp_cort(path)
#     print(path + ' cort done')
#     applywarp_subcort(path)
#     print(path + ' subcort done')

#     from joblib import Parallel, delayed
# Parallel(n_jobs=10)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))




[None, None, None, None, None, None, None]

In [1]:
import subprocess
import os
import time
import glob

from joblib import Parallel, delayed


# /mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat
# directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'  #to practice on how pipeline will run with multiple images 
# /mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat
directory = '/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat'  #to practice on how pipeline will run with multiple images 


def invwarp(directory):
    return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)


def applywarp_cort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

def applywarp_subcort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)


def apply_warps(path):
    
    folder = os.path.join(path, "Warps") 
    print(path)
    # if not os.path.exists(folder):
    #     os.mkdir(folder)

    invwarp(path)
    print(path + ' invwarp done')
    time.sleep(3)
    applywarp_cort(path)
    print(path + ' cort done')
    applywarp_subcort(path)
    print(path + ' subcort done')

apply_warps(os.path.join(directory))

#     from joblib import Parallel, delayed
# Parallel(n_jobs=10)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))




/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat


In [17]:
ls -l directory

ls: cannot access 'directory': No such file or directory


In [16]:
def invwarp(directory):

    print(directory+"/T1.nii.gz")
    return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)

directory= '/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat'

invwarp(directory)

/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat/T1.nii.gz


CompletedProcess(args=['invwarp', '--ref=/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat/T1.nii.gz', '--warp=/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat/T1_to_MNI_nonlin_field.nii.gz', '--out=/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat/Warps/std_subject_space', '--verbose'], returncode=134, stdout=b'Reading input files\nInverting field\n\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 1\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 2\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 3\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 4\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 5\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 6\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 7\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 8\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 9\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08Slice: 10\x08\x08\x08\x08\x0

In [ ]:
def applywarp_cort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

directory= '/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat'

applywarp_cort(directory)

In [ ]:
def applywarp_subcort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

directory= '/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat'

invwarp(directory)

In [14]:
print(directory+"/Warps/"+"std_subject_space.nii.gz")

/mnt/md0/cads-phd/explainbrainROI/run_dir/ca/sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.anat/Warps/std_subject_space.nii.gz


In [1]:
# fslstats -K subcort_HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the subcortical regions
# fslstats -K HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the cortical regions


def get_vols(vol_files, label):
    
    patient = {}
    
    for file in vol_files:
        
        path = os.path.basename(file)
        
        label_vals = [10, 11, 12, 13, 16, 17, 18, 26, 49, 50, 51, 52, 53, 54, 58]
        length = len(label_vals)

        vols = {}
        
        
        for i in label_vals:
            out = sout = subprocess.Popen(["fslstats", file, "-l", str((i+0.5)-1), "-u", str(i+0.5), "-V"], 
                   stdout=subprocess.PIPE, 
                   stderr=subprocess.STDOUT)
            stdout,stderr = out.communicate()
            vols["LABEL"] = label
            vols[str(i)] = [float(str(stdout.split()[0])[2:-1]), float(str(stdout.split()[1])[2:-1])]

        patient[path[:-29]] = vols
    
    return patient

In [ ]:
# fslstats -K subcort_HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the subcortical regions
# fslstats -K HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the cortical regions

# I need to
# 1. feed in the parent directory
# 2. feed in the label for each file and where the file is located
# 3. apply fslstats for cort and applywarp_subcort
# 4. collect the outputs and feed into dictionary
    



def getVol_Dataframe(directory):

    brain_volumes = pd.DataFrame()

    for file in glob.glob(os.path.join(directory)):

        path = os.path.basename(file)

        print(file)

        # vols = {}

        # for i in label_vals:
        #     print(i)
        #     out_cort = sout = subprocess.Popen(["fslstats", "-K", "./Warps/HO_in_subj_t1_space.nii.gz", "T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        #     stdout,stderr = out.communicate()
        #     out_subcort = sout = subprocess.Popen(["fslstats", "-K", "./Warps/subcort_HO_in_subj_t1_space.nii.gz", "T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        #     stdout,stderr = out.communicate()

        #     vols["LABEL"] = label
        #     vols[str(i)] = [float(str(stdout.split()[0])[2:-1]), float(str(stdout.split()[1])[2:-1])]

        # patient[path[:-29]] = vols

In [24]:
pwd

'/mnt/md0/cads-phd/explainbrainROI'

In [3]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed



directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'  #to practice on how pipeline will run with multiple images 


# for file in vol_files:
    
#     path = os.path.basename(file)

# directory = '/path/to/your/directory/*'  # Replace with your directory path

for file in glob.glob(directory):
    print(f"Full Path: {file}")
    
    filename = os.path.basename(file)
    print(f"Filename: {filename}")

Full Path: /mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/
Filename: 


In [3]:
import os 
import subprocess


directory =  "/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again"



for filename in os.listdir(directory):
    if filename.endswith(".anat"):


        #  Cortical FSL Stats Volume Extraction
        print(filename+"/Warps/HO_in_subj_t1_space.nii.gz")
        print(filename+"/T1.nii.gz")
        # out_cort = sout = subprocess.Popen(["fslstats", "-K", "./"+filename+"/Warps/HO_in_subj_t1_space.nii.gz", "./"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        print("Cortical FSL Stats Volume Extraction")
        out_cort = sout = subprocess.Popen(["fslstats", "-K",directory+"/"+filename+"/Warps/HO_in_subj_t1_space.nii.gz", directory+"/"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

        cort_stdout,stderr = out_cort.communicate()
        print(cort_stdout)
        print("NEW***************************************************************************************")

        # Subcortical FSL Stats Volume Extraction
        print(filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz")
        print(filename+"/T1.nii.gz")
        print("Subcortical FSL Stats Volume Extraction")
        out_subcort = sout = subprocess.Popen(["fslstats", "-K", directory+"/"+filename+"/Warps/subcort_HO_in_subj_t1_space.nii.gz", directory+"/"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        subcort_stdout,stderr = out_subcort.communicate()
        print(subcort_stdout)
        print("NEW***************************************************************************************")


        # print(filename+"/Warps/HO_in_subj_t1_space.nii.gz")
        # print(filename+"/T1.nii.gz")
        # out_cort = sout = subprocess.Popen(["fslstats", "-K", "./"+filename+"/Warps/HO_in_subj_t1_space.nii.gz", "./"+filename+"/T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        # cort_stdout,stderr = out_cort.communicate()
        # print(cort_stdout)
        # print("NEW***************************************************************************************")
        
        

sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/Warps/HO_in_subj_t1_space.nii.gz
sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.anat/T1.nii.gz
Cortical FSL Stats Volume Extraction
b'66547 66546.670780 \n7923 7922.960803 \n9000 8999.955475 \n11364 11363.943780 \n1713 1712.991525 \n3463 3462.982868 \n13793 13792.931763 \n18981 18980.906097 \n598 597.997042 \n4517 4516.977654 \n1329 1328.993425 \n4473 4472.977871 \n4182 4181.979311 \n1369 1368.993227 \n2490 2489.987682 \n3018 3017.985069 \n12625 12624.937542 \n5685 5684.971875 \n3518 3517.982596 \n2818 2817.986059 \n5530 5529.972642 \n37560 37559.814184 \n11491 11490.943152 \n2566 2565.987306 \n2848 2847.985910 \n6331 6330.968679 \n6762 6761.966547 \n13346 13345.933975 \n10235 10234.949366 \n8551 8550.957697 \n22142 22141.890459 \n1866 1865.990769 \n13134 13133.935024 \n3899 3898.980711 \n1873 1872.990734 \n9964 9963.950706 \n1145 1144.994335 \n4487 4486.977802 \n4550 4549.977490 \n4415 4414.978158 \n1524 1523.992460 \n7060 7059

In [11]:
pwd

'/mnt/md0/cads-phd/explainbrainROI'

In [1]:
import nibabel as nib
# /mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.anat/T1.nii.gz
# directory =  "/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again" 
# img_path = os.path.join(directory, "sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.nii.anat/T1.nii.gz")
img = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.anat/T1.nii.gz'


# "./"+filename+"/T1.nii.gz"
# print(img)
img = nib.load(img)
print(type(img))
img_data = img.get_fdata()
img_data.shape

<class 'nibabel.nifti1.Nifti1Image'>


(192, 256, 170)